In [30]:
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
#from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMChain, SimpleSequentialChain


### Comom parameters

When using both OpenAI and Google models, there is a parameter called "temperature," which is, in a way, related to the creativity of the response. The value of "temperature" ranges from 0 to 1. The lower the value, the more deterministic and predictable the response will be. Conversely, the higher the value, the more creative and surprising the response can be.
### API Keys

To function properly, both the Google and OpenAI models require specific API keys. These API keys are essential for accessing the models' features and have been set up as environment variables. For the Google model, the key is named HUGGINGFACEHUB_API_TOKEN, and for the OpenAI model, the key is termed OPENAI_API_KEY.















## Load API Keys as env variables

In [24]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

# 1. Generating results from an LLM based on Open AI Chat GPT

#### In the example below, we will generate just one result

In [25]:
llm_openai = OpenAI(temperature = 0.85)
prompt = "What would a good company name for be a company that makes colorful socks?"
print(llm_openai(prompt))



"Rainbow Socks Co."


#### In the exemple below, er will generate 5 differents results

In [26]:
result = llm_openai.generate([prompt]*5)
for company_name in result.generations:
    print(company_name[0].text)



"Rainbow Toes Co."


Rainbow Socks Co. 
 

Rainbow Socks Co. 

RainbowFootwear


"Rainbow Socks Co."


# 2. Generating results from an LLM based on Google

In the following example, we are utilizing the 'google/flan-t5-base' model, which is available in the Hugging Face library. It's crucial to clarify that the chosen model is merely for illustrative purposes. Alternatively, we could employ other models such as 'bert-base-uncased', 'gpt-2', 'roberta-large', or 'distilbert-base-uncased'.

In [27]:
llm_google = HuggingFaceHub(repo_id="google/flan-t5-base", 
                            model_kwargs={"temperature":0, "max_length":64})
prompt = "What are good fitness tips?"
print(llm_google(prompt))

Exercise for at least 30 minutes a day.


# 3. Prompt Templating and Chaining

Below, we feature a snippet of code that establishes a prompt template for a consultant specializing in devising names for new businesses. The purpose of this prompt is to enable it to suggest an array of business name options adaptable to any specified type of product.





In [28]:
template = "You are a namining consultant for new companies. What is a good name for a {company} that makes {product}?"

prompt = PromptTemplate.from_template(template)
llm_prompt = OpenAI(temperature=0.9)

chain = LLMChain(llm = llm_prompt, prompt = prompt)

In [29]:
print(chain.run({"company":"ABC Startup", "product": "colorful socks"}))



"Vibrant Sox Co."


### 3.1 - Simple Sequential Chains

In [32]:
llm_simple_sequential_chain = OpenAI(temperature = 0)
template = "What is a good name for a company that makes {product}"
first_prompt = PromptTemplate.from_template(template)
first_chain = LLMChain(llm=llm_simple_sequential_chain, prompt = first_prompt)

print(first_chain.run("colorful socks"))



"Rainbow Threads" or "Vibrant Soles"


In [33]:
second_template = "Write a catch phrase for the following company: {company_name}"
second_prompt = PromptTemplate.from_template(second_template)
second_chain = LLMChain(llm=llm_simple_sequential_chain, prompt = second_prompt)

#### 3.1.1 - Now, we have two chains. The first generates a company name suggestion, and the second creates a slogan. By employing a simple sequential chain, we can use the output of the first chain as the input for the second.

In the provided example, we have just two sequential chains, but envision the myriad possibilities when chaining multiple prompts towards a specific goal.

In the example below, we initiate a sequential chain where the product name intended for sale is given as input. Based on the product name, the Large Language Model suggests a company name, and then, drawing from the suggested company name, it proposes two slogans.

In [35]:
overall_chain = SimpleSequentialChain(chains=[first_chain, second_chain], verbose = True)

In [36]:
catchphrase = overall_chain.run("colorfull socks")



> Entering new SimpleSequentialChain chain...


"Rainbow Threads" or "Vibrant Soles"


"Color your world with Rainbow Threads" 
"Step into style with Vibrant Soles"

> Finished chain.
